<a href="https://colab.research.google.com/github/Alwyn25/Resumeparsing/blob/main/NER_RESUME_PARSER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
e

In [3]:
import spacy  # version 3.5.0
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.5.0'

In [5]:
!nvidia-smi

Fri Jan 27 06:02:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    31W /  70W |    334MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# !git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git
!git clone https://github.com/Alwyn25/Resumeparsing.git

Cloning into 'Resumeparsing'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), 5.30 MiB | 7.16 MiB/s, done.


In [7]:
cv_data = json.load(open('/content/Resumeparsing/data/training/train_data.json', 'r'))

In [8]:
len(cv_data)

200

In [9]:
!python -m spacy init fill-config /content/Resumeparsing/data/training/base_config.cfg  /content/Resumeparsing/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/Resumeparsing/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [11]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db
      

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

In [13]:
len(train), len(test)

(160, 40)

In [14]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 40/40 [00:00<00:00, 97.59it/s]


In [15]:
len(db.tokens)

40

In [16]:
!python -m spacy train /content/Resumeparsing/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-01-27 06:02:31,033] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-01-27 06:02:31,045] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2023-01-27 06:02:31,049] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2023-01-27 06:02:31,050] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassifi

In [17]:
#### MODEL TEST

In [18]:
nlp = spacy.load('/content/output/model-best')

In [19]:
doc = nlp(""""ENGINEERING LAB TECHNICIAN
Career Focus
My main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my
skills and continue to gain experience in the aerospace industry to advance in my career.
Professional Experience
Engineering Lab Technician Oct 2016 to Current
Company Name ï¼​ City , State
Responsible for testing various seat structures to meet specific certification requirements. Â
Maintain and calibrate test instruments to ensure testing capabilities are maintained.
Ensure data is captured and recorded correctly for certification test reports.
Duties also dynamic test set-up and static suite testing.
Engineering Lab Technician, Sr. Specialist Apr 2012 to Oct 2016
Company Name ï¼​ City , State
Utilized skills learned from LabView Course 1 training to construct and maintain LabView VI programs.
Responsible for fabricating and maintaining hydraulic/electrical test equipment to complete development and qualification programs.
Apply engineering principles to operate electrical, mechanical, and hydraulic systems to test pumps, motors, and actuators for aircraft.
Work closely with Design Engineers and Lab Management to investigate performance/design issues.
Completed Lean Six Sigma/5s training seminar.
Assembly/ Test Technician Mar 2007 to Mar 2012
Company Name ï¼​ City , State
Lead technician overseeing three member crew responsible for maintaining high quality standards in a high volume production environment.
Responsible for cargo door system actuators for Boeing 787 aircraft.
Experience with hydraulic components that meet military and FAA regulations.
Work closely with Engineering Group and Quality Assurance inspectors to resolve problems with development actuators.""""")
for ent in doc.ents:
  print(ent.text, "   ->>>>> ", ent.label_)

In [22]:
doc = nlp('my name is Laxmi Kant Tiwari. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
    print(ent.text, "   ->>>>> ", ent.label_)

Laxmi Kant Tiwari    ->>>>>  Name
Microsoft    ->>>>>  Companies worked at


In [23]:
print(doc.ents,ent.label_)

(Laxmi Kant Tiwari, Microsoft) Companies worked at


In [24]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 84.8 MB/s eta 0:00:00


In [25]:
import sys, fitz

In [28]:
fname = '/content/Resumeparsing/data/test/Smith Resume.pdf'
doc = fitz.open(fname)

In [29]:
 #doc = [page.getText() for page in doc]

In [30]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [31]:
text =""""ENGINEERING LAB TECHNICIAN
Career Focus
My main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my
skills and continue to gain experience in the aerospace industry to advance in my career.
Professional Experience
Engineering Lab Technician Oct 2016 to Current
Company Name ï¼​ City , State
Responsible for testing various seat structures to meet specific certification requirements. Â
Maintain and calibrate test instruments to ensure testing capabilities are maintained.
Ensure data is captured and recorded correctly for certification test reports.
Duties also dynamic test set-up and static suite testing.
Engineering Lab Technician, Sr. Specialist Apr 2012 to Oct 2016
Company Name ï¼​ City , State
Utilized skills learned from LabView Course 1 training to construct and maintain LabView VI programs.
Responsible for fabricating and maintaining hydraulic/electrical test equipment to complete development and qualification programs.
Apply engineering principles to operate electrical, mechanical, and hydraulic systems to test pumps, motors, and actuators for aircraft.
Work closely with Design Engineers and Lab Management to investigate performance/design issues.
Completed Lean Six Sigma/5s training seminar.
Assembly/ Test Technician Mar 2007 to Mar 2012
Company Name ï¼​ City , State
Lead technician overseeing three member crew responsible for maintaining high quality standards in a high volume production environment.
Responsible for cargo door system actuators for Boeing 787 aircraft.
Experience with hydraulic components that meet military and FAA regulations.
Work closely with Engineering Group and Quality Assurance inspectors to resolve problems with development actuators."""""
text


'"ENGINEERING LAB TECHNICIAN\nCareer Focus\nMy main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my\nskills and continue to gain experience in the aerospace industry to advance in my career.\nProfessional Experience\nEngineering Lab Technician Oct 2016 to Current\nCompany Name ï¼\u200b City , State\nResponsible for testing various seat structures to meet specific certification requirements. Â\nMaintain and calibrate test instruments to ensure testing capabilities are maintained.\nEnsure data is captured and recorded correctly for certification test reports.\nDuties also dynamic test set-up and static suite testing.\nEngineering Lab Technician, Sr. Specialist Apr 2012 to Oct 2016\nCompany Name ï¼\u200b City , State\nUtilized skills learned from LabView Course 1 training to construct and maintain LabView VI programs.\nResponsible for fabricating and maintaining hydraulic/electrical test equipment to comple

In [32]:
text = text.strip()

In [33]:
text = ' '.join(text.split())

In [34]:
text

'"ENGINEERING LAB TECHNICIAN Career Focus My main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my skills and continue to gain experience in the aerospace industry to advance in my career. Professional Experience Engineering Lab Technician Oct 2016 to Current Company Name ï¼\u200b City , State Responsible for testing various seat structures to meet specific certification requirements. Â Maintain and calibrate test instruments to ensure testing capabilities are maintained. Ensure data is captured and recorded correctly for certification test reports. Duties also dynamic test set-up and static suite testing. Engineering Lab Technician, Sr. Specialist Apr 2012 to Oct 2016 Company Name ï¼\u200b City , State Utilized skills learned from LabView Course 1 training to construct and maintain LabView VI programs. Responsible for fabricating and maintaining hydraulic/electrical test equipment to complete development

In [35]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "   ------->>>>> ", ent.label_)

Engineering Lab Technician    ------->>>>>  Designation


In [37]:
!zip -r /content/Resumeparsing.zip /content/Resumeparsing

  adding: content/Resumeparsing/ (stored 0%)
  adding: content/Resumeparsing/Resume_Parsing_with_Spacy.ipynb (deflated 75%)
  adding: content/Resumeparsing/README.md (stored 0%)
  adding: content/Resumeparsing/.git/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/heads/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/heads/main (stored 0%)
  adding: content/Resumeparsing/.git/refs/tags/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/remotes/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/remotes/origin/ (stored 0%)
  adding: content/Resumeparsing/.git/refs/remotes/origin/HEAD (stored 0%)
  adding: content/Resumeparsing/.git/branches/ (stored 0%)
  adding: content/Resumeparsing/.git/info/ (stored 0%)
  adding: content/Resumeparsing/.git/info/exclude (deflated 28%)
  adding: content/Resumeparsing/.git/HEAD (stored 0%)
  adding: content/Resumeparsing/.git/objects/ (stored 0%)
  adding: content/Resumeparsing/.

In [ ]:
!zip -r /content.zip /content

In [38]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-best/ (stored 0%)
  adding: content/output/model-best/ner/ (stored 0%)
  adding: content/output/model-best/ner/model (deflated 8%)
  adding: content/output/model-best/ner/moves (deflated 74%)
  adding: content/output/model-best/ner/cfg (deflated 33%)
  adding: content/output/model-best/meta.json (deflated 66%)
  adding: content/output/model-best/config.cfg (deflated 61%)
  adding: content/output/model-best/vocab/ (stored 0%)
  adding: content/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-best/vocab/strings.json (deflated 76%)
  adding: content/output/model-best/vocab/vectors (deflated 45%)
  adding: content/output/model-best/vocab/lookups.bin (stored 0%)
  adding: content/output/model-best/vocab/key2row (stored 0%)
  adding: content/output/model-best/tokenizer (deflated 81%)
  adding: content/output/model-best/transformer/ (stored 0%)
  adding: content/output/model-best/transformer/mod

In [ ]:
!zip -r /content/error.txt.zip /content/error.txt
!zip -r /content/test_data.spacy.zip /content/test_data.spacy
!zip -r /content/train_data.spacy.zip /content/train_data.spacy